We will compute the stimulus differentiation for all stimuli used in the experiments in this notebook.  
Descriptions fo stimuli can be found here: https://brainmapportal-live-4cc80a57cd6e400d854-f7fdcae.divio-media.net/filer_public/0f/5d/0f5d22c9-f8f6-428c-9f7a-2983631e72b4/neuropixels_cheat_sheet_nov_2019.pdf

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

from os import path
from glob import glob
import pickle

import numpy as np
import pandas as pd
import scipy as sp
from skimage.transform import resize
import imageio
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import patches
from IPython.display import display, Video
from IPython.utils.capture import capture_output
from tqdm.auto import tqdm
with capture_output():
    tqdm.pandas()

import differentiation

from ipympl.backend_nbagg import Canvas
Canvas.header_visible.default_value = False

In [2]:
stimulus_directory = (
    '/allen/programs/braintv/workgroups/'
    'tiny-blue-dot/differentiation/refactor/stimuli'
)
frame_size = (130, 192)
frame_rate = 200 # in Hz; same as what we use with the firing rates
s_freq_scale = 4

In [3]:
filenames = {
    'flashes' : 'stim_flash_90',
    'gabors' : 'stim_gabors_90',
    'drifting_gratings' : 'stim_dg_90',
    'drifting_gratings_contrast' : 'stim_dgc_90',
    'static_gratings' : 'stim_sg_90',
    'natural_movie_one' : 'stim_natural_movie_one',
    'natural_movie_one_shuffled' : 'stim_natural_movie_one_shuffled',
    'natural_movie_three' : 'stim_natural_movie_three'
}

def dict_reversed(dic):
    return {v:k for k, v in dic.items()}

In [4]:
stim_colors = {
    'spontaneous' : cm.Greys(0.3, 0.8),
    'gabors' : cm.Reds(0.3, 0.8),
    'flashes' : cm.Reds(0.8, 0.8),
    'drifting_gratings' : cm.Blues(0.5, 0.8),
    'drifting_gratings_contrast' : cm.Blues(0.7, 0.8),
    'static_gratings' : cm.Blues(0.9, 0.8),
    'natural_movie_three' : cm.Greens(0.75, 0.8),
    'natural_movie_two' : cm.Greens(0.6, 0.8),
    'natural_movie_one' : cm.Greens(0.45, 0.8),
    'natural_movie_one_shuffled' : cm.Purples(0.3, 0.8),
    'Spontaneous' : cm.Greys(0.6, 0.8),
    'Artificial (simple)' : cm.Reds(0.6, 0.8),
    'Artificial (complex)' : cm.Blues(0.8, 0.8),
    'Natural' : cm.Greens(0.8, 0.8),
}

# Preview stimulus clips

In [4]:
movies = glob(path.join(stimulus_directory, 'stim_*.mp4'))
for movie in movies:
    display(Video(movie))

# Understanding how differentiation scales with state length

In [5]:
def _mkplt(data, window, state_length, norm=None, state_length_normalization=False):
    states = differentiation.spectral_states(frame_rate, state_length, data, norm=norm)
    diff = differentiation.spectral_differentiation(data, window_length=state_length, sample_rate=frame_rate, norm=norm)
    
    x = np.ones(3000)[np.newaxis, :]
    y = differentiation.spectral_states(frame_rate, state_length, x, norm=norm)[0, 0]
    print('For an "all ones" vector, spectral state magnitude is:', end=' ')
    print(y)
    # normalize with this number?
    if state_length_normalization:
        diff = diff / y
        states = states / y
    
    # and plot it
    f = plt.figure(figsize=(3, 4), constrained_layout=True)
    spec = plt.GridSpec(3, 2, f, width_ratios=[19, 1], height_ratios=[1, 1, 3], hspace=0.04)
    ax_sig = f.add_subplot(spec[0, 0])
    ax_states = f.add_subplot(spec[1, 0], sharex=ax_sig)
    ax_dist = f.add_subplot(spec[2, 0], sharex=ax_sig)
    ax_states_cbar = f.add_subplot(spec[1, 1])
    ax_dist_cbar = f.add_subplot(spec[2, 1])

    ax_sig.plot(np.arange(0, *np.diff(window), 1/frame_rate), data[0])
    ax_sig.set_ylabel('intensity')
    try:
        ax_sig.set_title(f'Activity of a single pixel ({stimulus}, state length {state_length}s)')
    except:
        pass

    for i in np.arange(0, *np.diff(window), 1/frame_rate)[::int(state_length*frame_rate)]:
        ax_sig.axvline(i, c='k', lw=0.2)

    im = ax_states.imshow(
        states.T[:10, :], aspect='auto', interpolation='none',
        extent=[0, np.diff(window)[0], *np.fft.rfftfreq(int(state_length*frame_rate), 1/frame_rate)[[-1, 0]]], cmap=cm.Purples
    )
    plt.colorbar(im, cax=ax_states_cbar, label='power')
    ax_states.set_ylabel('frequency')
    ax_states.set_title(f'differentiation = {np.median(diff):.0f}')

    im = ax_dist.imshow(
        sp.spatial.distance.squareform(diff), aspect='auto', interpolation='none',
        extent=[0, np.diff(window)[0], np.diff(window)[0], 0], cmap=cm.Oranges
    )
    plt.colorbar(im, cax=ax_dist_cbar, label='distance')
    ax_dist.set_ylabel('time (s)')
    ax_dist.set_xlabel('time (s)')

    ax_sig.tick_params(bottom=False, labelbottom=False)
    ax_states.tick_params(bottom=False, labelbottom=False)
    
    return data, states, diff, f

## Spectrum magnitude changes with state length even with constant signal

In [6]:
# constant signal
window = np.array([0, 90])
data = np.ones((1, np.diff(window)[0]*frame_rate))
norm = None
win_length = 3
state_lengths = [0.03, 0.06, 0.1, 0.15, 0.3, 0.6, 1, 3, 6, 10, 15, 30, 45]
mag = []
for state_length in state_lengths:
    mag.append(differentiation.spectral_states(frame_rate, state_length, data, norm=norm)[0, 0])

f, ax = plt.subplots(figsize=(4, 3), tight_layout=True)
ax.plot(state_lengths, mag, '-o')
ax.set_xlabel('state length (s)')
ax.set_ylabel('0-freq bin magnitude')
ax.set_xscale('log')
ax.set_yscale('log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

This is reflected in the differentiation, which also scales as a powerlaw with state length for piecewise constant signal

### Piecewise constant stimulus

In [7]:
# piecewise constant signal
norm = None
window = np.array([0, 90])
_typ_len = 10
data = np.random.randint(0, 255, (100, 1, int(np.diff(window)[0]/_typ_len))).repeat(_typ_len*frame_rate, axis=2) # 100 trials, 1 neuron

diff = {}
state_lengths = [0.03, 0.06, 0.1, 0.15, 0.3, 0.6, 1, 3, 6, 10, 15, 30, 45]
for state_length in tqdm(state_lengths):
    diff[state_length] = np.median(differentiation.spectral_differentiation(data, window_length=state_length, sample_rate=frame_rate, norm=norm), axis=1)

diff = pd.DataFrame(diff)

f, (ax0, ax) = plt.subplots(2, 1, figsize=(4, 4), tight_layout=True, gridspec_kw=dict(height_ratios=[1, 3]))

ax0.plot(np.linspace(*window, data.shape[-1], False), data[0, 0, :])
ax0.set_ylabel('signal')
ax0.set_xlabel('time (s)')

diff.mean().plot(yerr=diff.std(), ax=ax, label='_')

m, c = np.polyfit(*np.log(diff.mean().reset_index()).values.T, 1)
_x = [1, 2, 3]
_y = np.exp(m*np.log(_x)+c)/2
ax.plot(_x, _y, c='r', label=f'alpha={m:.2f}')

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('state length')
ax.set_ylabel('differentiation')
ax.legend(fontsize=8)

for ax in [ax0, ax]:
    sns.despine(ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

We can remove this dependence by scaling the differentiation to account for changes in the normalization of the FFT. In this case, the differentiation remains flat for all state lengths that are smaller than the piecewise constant length. For longer state lengths, obviously, differentiation will start changing.

In [8]:
# piecewise constant signal
norm = None
window = np.array([0, 90])
_typ_len = 10
data = np.random.randint(0, 255, (100, 1, int(np.diff(window)[0]/_typ_len))).repeat(_typ_len*frame_rate, axis=2)

diff = {}
state_lengths = [0.03, 0.06, 0.1, 0.15, 0.3, 0.6, 1, 3, 6, 10, 15, 30, 45]
for state_length in tqdm(state_lengths):
    # SCALING FACTOR
    diff[state_length] = np.median(differentiation.spectral_differentiation(data, window_length=state_length, sample_rate=frame_rate, norm=norm), axis=1)/state_length**2

diff = pd.DataFrame(diff)

f, (ax0, ax) = plt.subplots(2, 1, figsize=(4, 4), tight_layout=True, gridspec_kw=dict(height_ratios=[1, 3]))

ax0.plot(np.linspace(*window, data.shape[-1], False), data[0, 0, :])
ax0.set_ylabel('signal')
ax0.set_xlabel('time (s)')

diff.mean().plot(yerr=diff.std(), ax=ax, label='_', fmt='o')

m, c = np.polyfit(*np.log(diff.mean().reset_index()).values.T, 1)
_x = state_lengths
_y = np.exp(m*np.log(_x)+c)
ax.plot(_x, _y, c='r', label=f'slope={m:.2f}')

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('state length')
ax.set_ylabel('differentiation')
ax.legend(fontsize=8, loc=3, frameon=False)

for ax in [ax0, ax]:
    sns.despine(ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Assuming we are in the regime where state lengths are longer than the timescales over which the signals change (in the brain, we expect that the signals change over a few hundreds of milliseconds, so we are talking about states longer than ~1s), how does differentiation scale with state size?  
Here we will look at differentiation for white noise. A good metric of differentiation should be independent of the state length for white noise, since the complexity is constant irrespective of the timescale.

## Spectral differentiation for white noise

In [9]:
# white noise differentiation should be constant (at least in the absence of discretization artifacts)
norm = None
data = np.random.randint(0, 255, (100, 1, 9000))

diff = {}
state_lengths = [0.03, 0.06, 0.1, 0.15, 0.3, 0.6, 1, 3, 9, 15]
for state_length in tqdm(state_lengths):
    diff[state_length] = np.median(differentiation.spectral_differentiation(data, window_length=state_length, sample_rate=frame_rate, norm=norm)/state_length**2, axis=1)

diff = pd.DataFrame(diff)

f, (ax0, ax) = plt.subplots(2, 1, figsize=(4, 4), tight_layout=True, gridspec_kw=dict(height_ratios=[1, 3]))

ax0.plot(np.linspace(*window, data.shape[-1], False), data[0, 0, :])
ax0.set_ylabel('signal')
ax0.set_xlabel('time (s)')

diff.mean().plot(yerr=diff.std(), ax=ax, label='_', fmt='o')

m, c = np.polyfit(*np.log(diff.mean().reset_index()).values.T, 1)
_x = state_lengths
_y = np.exp(m*np.log(_x)+c)
ax.plot(_x, _y, c='r', label=f'slope={m:.2f}')

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('state length')
ax.set_ylabel('differentiation')
ax.legend(fontsize=8, loc=3, frameon=False)

for ax in [ax0, ax]:
    sns.despine(ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

But we see that here, the differentiation still scales with state length. To explore this, let us look at three state lengths, and the spectra we get for them:

In [10]:
# # try for white noise data that should have a flat spectrum
# window = np.array([0, 90])
# states = {}
# diffs = {}
# for state_length in [0.03, 0.3, 3]:
#     data = np.random.randint(0, 255, (1, np.diff(window)[0]*frame_rate))
#     data, states[state_length], diffs[state_length], f = _mkplt(data, window, state_length, norm='ortho', state_length_normalization=True)

In [11]:
# # plot spectra in each state
# f, ax = plt.subplots(figsize=(4, 3), tight_layout=True)
# for state, cmap in zip(states.keys(), [cm.Reds, cm.Greens, cm.Blues]):
#     ax.plot(np.fft.rfftfreq(int(state*frame_rate), 1/frame_rate), states[state].T, c=cmap(0.4, 0.1))
#     ax.plot(np.fft.rfftfreq(int(state*frame_rate), 1/frame_rate), states[state].mean(0), c=cmap(0.99, 0.99), label=state)
#     ax.set_yscale('log')
#     ax.set_xlabel('frequency')
#     ax.set_ylabel('power')
# ax.legend(fontsize=8, title='state length (s)', title_fontsize=9);

Which frequencies contribute most to differentiation? The 0-frequency bin! So the change in mean signal due to changing bin size drives a decrease in differentiation. Because we are averaging out the signal more for larger states, the differentiation magnitude decreases.

In [12]:
# f, (ax2, ax) = plt.subplots(1, 2, figsize=(6, 3), tight_layout=True, sharey=True)#, gridspec_kw=dict(width_ratios=[2, 1]))
# ax.scatter(*np.array([[sl, sp[:, 0].std()] for sl, sp in states.items()]).T)
# ax.set_xlabel('state length (s)')
# ax.set_xscale('log')
# ax.set_yscale('log')

# ax2.scatter(*np.array([[np.median(diffs[sl]), np.median(
#     sp.spatial.distance.pdist(states[sl][:, [0]], metric='euclidean')
# )] for sl in diffs.keys()]).T, label='bin 0')

# ax2.scatter(*np.array([[np.median(diffs[sl]), np.median(
#     sp.spatial.distance.pdist(states[sl][:, [1]], metric='euclidean')
# )] for sl in diffs.keys()]).T, label='bin 1')

# # ax2.scatter(*np.array([[np.median(diffs[sl]), np.median(
# #     sp.spatial.distance.pdist(states[sl][:, [2]], metric='euclidean')
# # )] for sl in diffs.keys()]).T, label='bin 2')

# ax2.set_xlabel('differentiation')
# ax2.set_ylabel('distance within frequency bin')
# ax2.set_xscale('log')
# ax2.set_yscale('log')
# ax2.legend(fontsize=8);
# # ax2.set_ylim(ax2.get_xlim());

## Differentiation should be scaled differently at different timescales
Putting all of this together, we see that if the signal autocorrelation length is shorter than the state size, then the differentiation (when normalized for the total FFT power) decreases with state length as a powerlaw with exponent 0.5. The net normalization in this case is given by $df_{normalized} = df~~/~~state*1.5$. When the autocorrelation length of the signal is longer than the state length, differentiation remains constant with changing state length.

In [10]:
# piecewise constant signal
norm = None
window = np.array([0, 120])
_typ_len = 0.6
data_dense = np.random.randint(
    0, 255, (
        100, 1, int(np.diff(window)[0]/_typ_len)
    )
).repeat(_typ_len*frame_rate, axis=2)

diff_dense = {}
state_lengths = [0.03, 0.06, 0.1, 0.15, 0.3, 0.6, 1, 3, 6, 10, 15, 30]
for state_length in tqdm(state_lengths):
    # SCALING FACTOR
    diff_dense[state_length] = np.median(
        differentiation.spectral_differentiation(
            data_dense, window_length=state_length,
            sample_rate=frame_rate, norm=norm
        ), axis=1
    )/state_length**2

diff_dense = pd.DataFrame(diff_dense)

f, (ax0, ax) = plt.subplots(
    2, 1, figsize=(4, 4), tight_layout=True,
    gridspec_kw=dict(height_ratios=[1, 3])
)

ax0.plot(np.linspace(*window, data_dense.shape[-1], False), data_dense[0, 0, :], lw=1)
ax0.set_ylabel('signal')
ax0.set_xlabel('time (s)')

diff_dense.mean().plot(yerr=diff_dense.sem(), ax=ax, label='_', fmt='o')

m, c = np.polyfit(*np.log(diff_dense.mean().reset_index()).values.T[:, :6], 1)
_x = [0.03, 0.06, 0.1, 0.5]
_y = np.exp(m*np.log(_x)+c)
ax.plot(_x, _y, c='r', label=f'slope={m:.2f}')

m, c = np.polyfit(*np.log(diff_dense.mean().reset_index()).values.T[:, -7:-2], 1)
_x = [0.6, 1, 3, 6, 10, 45, 60]
_y = np.exp(m*np.log(_x)+c)
ax.plot(_x, _y, c='C4', label=f'slope={m:.2f}')

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('state length')
ax.set_ylabel('differentiation')
# ax.legend(fontsize=8, frameon=False)

ax.axvline(_typ_len, c='k')
ax.annotate(
    'autocorrelation\ntimescale', (_typ_len*1.1, (diff.mean()-diff.std()).iloc[-1]),
    fontsize=10
)

for ax in [ax0, ax]:
    sns.despine(ax=ax)

# f.savefig('fig_')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Normalizing by the effective scaling factor of state_length^1.5:

---
Differentiation for multiple uncorrelated units

In [21]:
# piecewise constant signal
N_rep = 500
N = 1
norm = None
window = np.array([0, 60])
_typ_len = 0.3
# should firing onset times be correlated?
ac_corr = True
# sparsity level
sparsity = 0.7

_data = np.random.randint(0, 255, (
    N_rep, N, int(np.diff(window)[0]/_typ_len)
))

# sparsity
for j in range(N_rep):
    for i in range(N):
        _data[j, i, np.random.rand(_data.shape[2])<sparsity] = 0

_data = _data.repeat(_typ_len*frame_rate, axis=2)

data = np.zeros(_data.shape)
# uncorrelated units
if not ac_corr:
    # jitter is random for each unit and trial
    for i in range(N):
        for j in range(N_rep):
            dt = np.random.randint(_typ_len*frame_rate)
            data[j, i, :] = np.roll(_data[j, i, :], dt, axis=-1)
# correlated units
else:
    # jitter is random for each trial, but same for all units
    for j in range(N_rep):
        dt = np.random.randint(_typ_len*frame_rate)
        data[j, :, :] = np.roll(_data[j, :, :], dt, axis=-1)

diff = {}
state_lengths = [0.03, 0.06, 0.1, 0.15, 0.3, 0.6, 1, 3, 6, 10, 15, 30]
for state_length in tqdm(state_lengths):
    diff[state_length] = np.median(
        differentiation.spectral_differentiation(
            data, window_length=state_length, sample_rate=frame_rate, norm=norm
        ), axis=1
    )/state_length**2

diff = pd.DataFrame(diff)

f, (ax0, ax) = plt.subplots(
    2, 1, figsize=(4, 4), tight_layout=True,
    gridspec_kw=dict(height_ratios=[1, 3])
)

ax0.plot(np.linspace(*window, data.shape[-1], False), data[0, :10, :].T, lw=1)
ax0.set_ylabel('signal')
ax0.set_xlabel('time (s)')
ax0.set_yticks([])

diff.mean().plot(yerr=diff.std(), ax=ax, label='_', fmt='o')

m, c = np.polyfit(*np.log(diff.mean().reset_index()).values.T[
    :, :sum(np.array(state_lengths)<_typ_len)
], 1)
_x = np.array(state_lengths)[np.array(state_lengths)<_typ_len]
_y = np.exp(m*np.log(_x)+c)
ax.plot(_x, _y, c='r', label=f'slope={m:.2f}')

m, c = np.polyfit(*np.log(diff.mean().reset_index()).values.T[
    :, -sum(np.array(state_lengths)>_typ_len):
], 1)
_x = np.array(state_lengths)[np.array(state_lengths)>_typ_len]
_y = np.exp(m*np.log(_x)+c)
ax.plot(_x, _y, c='C4', label=f'slope={m:.2f}')

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('state length')
ax.set_ylabel('differentiation')
# ax.legend(fontsize=8, frameon=False)
ax.set_yticks([])

ax.axvline(_typ_len, c='k')
ax.annotate(
    'autocorrelation\ntimescale',
    (_typ_len*1.1, (diff.mean()-diff.std()).iloc[-1]),
    fontsize=10
)

for ax in [ax0, ax]:
    sns.despine(ax=ax)

# f.suptitle(
#     f'{N} units, {N_rep} trials, sparsity: {sparsity}',# {"un" if not ac_corr else ""}correlated activity,
#     fontsize=9, color='r'
# );

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
f, (ax0, ax) = plt.subplots(
    2, 1, figsize=(3, 3),# tight_layout=True,
    gridspec_kw=dict(height_ratios=[1, 3])
)

ax0.plot(
    np.linspace(*window, data_dense.shape[-1], False),
    data_dense[0, 0, :], lw=1, c=cm.Greys(0.7, 0.8)
)
ax0.plot(
    np.linspace(*window, data.shape[-1], False)+0.01,
    data[0, :10, :].T, lw=0.75, c='k'
)
ax0.set_xlim(0, 20)

diff_dense = (diff_dense.T/diff_dense[1]).T
diff = (diff.T/diff[1]).T

diff_dense.mean().plot(yerr=diff_dense.sem(), ax=ax, label='dense signal', fmt='o', c=cm.Blues(0.4, 0.9))
diff.mean().plot(yerr=diff_dense.sem(), ax=ax, label='sparse signal', fmt='o', c=cm.Blues(0.8, 0.9))

m, c = np.polyfit(*np.log(diff_dense.mean().reset_index()).values.T[:, :6], 1)
_x = [0.03, 0.06, 0.1, 0.5]
_y = np.exp(m*np.log(_x)+c)
ax.plot(_x, _y, c=cm.Reds(0.4, 0.9))#, label=f'slope={m:.2f}')

m, c = np.polyfit(*np.log(diff.mean().reset_index()).values.T[
    :, :sum(np.array(state_lengths)<_typ_len)
], 1)
_x = np.array(state_lengths)[np.array(state_lengths)<=_typ_len]
_y = np.exp(m*np.log(_x)+c)
ax.plot(_x, _y, c=cm.Reds(0.8, 0.9))#, label=f'slope={m:.2f}')

m, c = np.polyfit(*np.log(diff_dense.mean().reset_index()).values.T[:, -7:-2], 1)
_x = [0.6, 1, 3, 6, 10, 45, 60]
_y = np.exp(m*np.log(_x)+c)
ax.plot(_x, _y, c=cm.Purples(0.4, 0.9))#, label=f'slope={m:.2f}')

m, c = np.polyfit(*np.log(diff.mean().reset_index()).values.T[
    :, -sum(np.array(state_lengths)>_typ_len):
], 1)
_x = np.array(state_lengths)[np.array(state_lengths)>_typ_len]
_y = np.exp(m*np.log(_x)+c)
ax.plot(_x, _y, c=cm.Purples(0.8, 0.9))#, label=f'slope={m:.2f}')

ax.set_xscale('log')
ax.set_yscale('log')

ax.set_xlabel('state length (s)', fontsize=8)
ax.set_ylabel('normalized\ndifferentiation', fontsize=8, labelpad=-3)
ax.legend(loc=0, fontsize=8, frameon=False)
ax.tick_params(axis='y', labelsize=7, pad=-1)
ax.tick_params(axis='x', labelsize=7, pad=1)

ax0.set_ylabel('signal', fontsize=8, labelpad=-3)
ax0.set_xlabel('time (s)', fontsize=8)
# ax0.set_yticks([])
ax0.tick_params(axis='y', labelsize=7, pad=-1)
ax0.tick_params(axis='x', labelsize=7, pad=1)

f.align_ylabels()

# ax.axvline(_typ_len, c='k')
# ax.annotate(
#     'autocorrelation\ntimescale', (_typ_len*1.1, (diff.mean()-diff.std()).iloc[-1]),
#     fontsize=10
# )
f.tight_layout(pad=0.9, h_pad=0.4)
# f.savefig('fig_timescales_toy.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

---

In [13]:
# piecewise constant signal
norm = None
window = np.array([0, 120])
_typ_len = 0.6
data = np.random.randint(0, 255, (100, 1, int(np.diff(window)[0]/_typ_len))).repeat(_typ_len*frame_rate, axis=2)

diff = {}
state_lengths = [0.03, 0.06, 0.1, 0.15, 0.3, 0.6, 1, 3, 6, 10, 15, 30, 40]
for state_length in tqdm(state_lengths):
    # SCALING FACTOR
    diff[state_length] = np.median(differentiation.spectral_differentiation(data, window_length=state_length, sample_rate=frame_rate, norm=norm), axis=1)/state_length**1.5

diff = pd.DataFrame(diff)

f, (ax0, ax) = plt.subplots(2, 1, figsize=(4, 4), tight_layout=True, gridspec_kw=dict(height_ratios=[1, 3]))

ax0.plot(np.linspace(*window, data.shape[-1], False), data[0, 0, :], lw=1)
ax0.set_ylabel('signal')
ax0.set_xlabel('time (s)')

diff.mean().plot(yerr=diff.std(), ax=ax, label='_')

m, c = np.polyfit(*np.log(diff.mean().reset_index()).values.T[:, :4], 1)
_x = [0.03, 0.06, 0.1, 0.5]
_y = np.exp(m*np.log(_x)+c)/2
ax.plot(_x, _y, c='r', label=f'alpha={m:.2f}')

m, c = np.polyfit(*np.log(diff.mean().reset_index()).values.T[:, -8:], 1)
_x = [1, 3, 6, 10, 45]
_y = np.exp(m*np.log(_x)+c)/2
ax.plot(_x, _y, c='C4', label=f'alpha={m:.2f}')

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('state length')
ax.set_ylabel('differentiation')
ax.legend(fontsize=8)

for ax in [ax0, ax]:
    sns.despine(ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# What are the autocorrelation timescales in the stimuli?

In [5]:
# compute autocorrelation for all stimuli (100 randomly selected pixels)
try:
    autocorr_times = pd.read_pickle(
        path.join(stimulus_directory, 'stimulus_autocorrelation.pkl')
    )
except:
    print(
        'Run firing_rate_autocorrelation.ipynb'
        ' stimulus autocorrelation section first.'
    )

In [7]:
autocorr_times = autocorr_times[autocorr_times.mean().sort_values().index]
f, ax = plt.subplots(figsize=(5, 1.8), tight_layout=True)
sns.boxplot(data=autocorr_times, linewidth=0.5, showfliers=False, color=cm.Greys(0.4, 0.4))
ax.set_xticklabels(
    [x.get_text().replace('_', '\n') for x in ax.get_xticklabels()],
    fontsize=7
)
ax.set_ylabel('characteristic\ntimescale (s)', fontsize=7.8)
ax.set_yticks([0, 0.5, 1, 1.5])
ax.tick_params(axis='both', labelsize=7);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

So we will stay in the constant differentiation regime for the last three stimuli unless the state size is made larger than 1s. For the rest, a state size of 0.5s should be closer to the optimal.  
Also, we can go to very small state sizes and normalize with state_length^2.

# How does differentiation vary with window length?

In [8]:
win_lengths = [0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6, 51.2]
state_length = 0.1
cohort_normalize = True

try:
    with open('stim_diffn_wrt_win_ln.pkl', 'rb') as f:
        diffn = pickle.load(f)
except:
    diffn = {}
    for stim in tqdm(filenames.keys()):
        if stim in diffn.keys():
            continue
        movie_data = np.load(path.join(stimulus_directory, f'{filenames[stim]}.npy'))
        diffn[stim] = {}
        for win_length in win_lengths:
            md_flattened = movie_data.reshape(movie_data.shape[0], np.prod(movie_data.shape[1:]))
            try:
                md_reshaped = np.reshape(md_flattened.T, (md_flattened.shape[1], -1, int(win_length*frame_rate))).transpose(1, 0, 2)
            except ValueError:
                md_flattened = md_flattened[:int(md_flattened.shape[0]/(win_length*frame_rate))*int(win_length*frame_rate)]
                md_reshaped = np.reshape(md_flattened.T, (md_flattened.shape[1], -1, int(win_length*frame_rate))).transpose(1, 0, 2)

            if cohort_normalize:
                # for cohort normalization
                md_reshaped = (md_reshaped.transpose(1, 2, 0) / md_reshaped.mean(axis=(-1, -2))).transpose(2, 0, 1)

            try:
                diff = differentiation.spectral_differentiation(md_reshaped, sample_rate=frame_rate, window_length=state_length)
                diffn[stim][win_length] = np.median(diff, axis=1)
            except:
                pass
    with open('stim_diffn_wrt_win_ln.pkl', 'wb') as f:
        pickle.dump(diffn, f)

In [9]:
diffn_mn = pd.concat({
    stim : pd.Series({
        k:v.mean() for k, v in diffn[stim].items()
    }, name=stim) for stim in diffn.keys()
}, axis=1)

diffn_sd = pd.concat({
    stim : pd.Series({
        k:v.std() for k, v in diffn[stim].items()
    }, name=stim) for stim in diffn.keys()
}, axis=1)

f, ax = plt.subplots(figsize=(6, 3), tight_layout=True)
diffn_mn.plot(ax=ax, yerr=diffn_sd)
ax.set_xscale('log')
ax.set_xlabel('window length (s)')
ax.set_ylabel('differentiation')
ax.legend(loc=(1.01, 0), fontsize=9);
f.savefig('fig_supp_diffn_vs_win_length_all_stimuli.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/saurabh.gandhi/Projects/.env_difflib/lib/python3.6/site-packages/numpy/core/_asarray.py:138: UserWarning: Warning: converting a masked element to nan.
  return array(a, dtype, copy=False, order=order, subok=True)


# Compute differentiation

In [29]:
cohort_normalize = True
try:
    diff_all = pd.read_pickle(path.join(stimulus_directory, f'spectral_differentiation_stimulus.pkl'))
    diff_all = diff_all.stack().to_dict()
except:
    diff_all = {}

for win_length in [0.3]:#[3, 9, 30]:
    for state_length in [0.03]:#[0.005, 0.01, 0.015, 0.02, 0.03, 0.06, 0.1, 0.15, 0.3, 0.5, 1, 3, 10]:
        if (win_length, state_length) in diff_all.keys():
            continue
        if win_length <= state_length:
            continue
        diffn = {}
        for stim, f in tqdm(filenames.items(), desc=f'{win_length} ({state_length})'):
            movie_data = np.load(path.join(stimulus_directory, f'{f}.npy'))
            md_flattened = movie_data.reshape(movie_data.shape[0], np.prod(movie_data.shape[1:]))
            try:
                md_reshaped = np.reshape(md_flattened.T, (md_flattened.shape[1], -1, int(win_length*frame_rate))).transpose(1, 0, 2)
            except ValueError:
                md_flattened = md_flattened[:int(md_flattened.shape[0]/(win_length*frame_rate))*int(win_length*frame_rate)]
                md_reshaped = np.reshape(md_flattened.T, (md_flattened.shape[1], -1, int(win_length*frame_rate))).transpose(1, 0, 2)

            if cohort_normalize:
                # for cohort normalization
                md_reshaped = (md_reshaped.transpose(1, 2, 0) / md_reshaped.mean(axis=(-1, -2))).transpose(2, 0, 1)

            diff = differentiation.spectral_differentiation(md_reshaped, sample_rate=frame_rate, window_length=state_length)
            diffn[stim] = np.median(diff, axis=1)
        diff_mn = {k:v.mean() for k, v in diffn.items()}
        diff_sd = {k:v.std() for k, v in diffn.items()}
        diff_all[(win_length, state_length)] = pd.concat([
            pd.Series(diff_mn).rename('mean_diff'),
            pd.Series(diff_sd).rename('std_diff')
        ], axis=1).sort_values('mean_diff').stack().to_dict()
diff_all = pd.DataFrame(diff_all).unstack()
diff_all.to_pickle(path.join(stimulus_directory, f'spectral_differentiation_stimulus.pkl'))

In [11]:
# diff_all.loc[:, (slice(None), 0.015)] = diff_all.loc[:, (slice(None), 0.015)]/4
# diff_all.loc[:, (slice(None), 0.02)] = diff_all.loc[:, (slice(None), 0.02)]/4
# diff_all.to_pickle(f'data/differentiation/spectral_differentiation/spectral_differentiation_stimulus.pkl')

In [32]:
# reorder with ascending differentiation under a 'standard' condition
diff_all = diff_all.loc[
    diff_all.xs('mean_diff', axis=1, level=2)\
    .xs(0.5, axis=1, level=1)\
    .mean(1).sort_values().index
]

# normalize by the number of pixels to compare with cells
diff_all = diff_all / np.sqrt(np.prod(frame_size))

# normalize by the scaling factor of state_length**1.5
diff_all = diff_all / diff_all.columns.get_level_values(1)**2

diff_all

3.0                                           \
                              0.005               0.010               0.015    
                           mean_diff  std_diff mean_diff  std_diff mean_diff   
drifting_gratings_contrast  0.000000  0.000000  0.000000  0.000000  0.000000   
flashes                     0.000000  0.000000  0.000000  0.000000  0.000000   
gabors                      0.072654  0.001091  0.072654  0.001091  0.072350   
drifting_gratings           0.476201  0.002943  0.471544  0.006901  0.464668   
natural_movie_one_shuffled  0.377102  0.002595  0.374721  0.002465  0.370624   
natural_movie_three         0.324211  0.084411  0.324118  0.084252  0.323915   
natural_movie_one           0.331199  0.076381  0.331255  0.076327  0.331196   
static_gratings             0.650261  0.007817  0.650261  0.007817  0.649960   

                                                                              \
                                        0.020               0.030              
                            std_diff mean_diff  std_diff mean_diff  std_diff   
drifting_gratings_contrast  0.000000  0.000000  0.000000  0.000000  0.000000   
flashes                     0.000000  0.000000  0.000000  0.000000  0.000000   
gabors                      0.001100  0.072350  0.001100  0.071983  0.001184   
drifting_gratings           0.016210  0.454904  0.029292  0.430358  0.062769   
natural_movie_one_shuffled  0.002590  0.364181  0.002770  0.345563  0.002645   
natural_movie_three         0.084009  0.323562  0.083677  0.322599  0.082924   
natural_movie_one           0.076171  0.331095  0.076126  0.330589  0.075770   
static_gratings             0.007777  0.649960  0.007777  0.649661  0.007736   

                            ...      30.0                                \
                            ...    0.500               1.000              
                            ... mean_diff  std_diff mean_diff  std_diff   
drifting_gratings_contrast  ...  0.017641  0.003467  0.020655  0.007465   
flashes                     ...  0.000000  0.000000  0.148989  0.003735   
gabors                      ...  0.050394  0.000158  0.035181  0.000629   
drifting_gratings           ...  0.072053  0.012360  0.063310  0.000002   
natural_movie_one_shuffled  ...  0.104737  0.000000  0.072970  0.000000   
natural_movie_three         ...  0.347038  0.011632  0.314442  0.013297   
natural_movie_one           ...  0.447099  0.000000  0.415360  0.000000   
static_gratings             ...  0.463615  0.001719  0.324646  0.000496   

                                                                    \
                              3.000               10.000             
                           mean_diff  std_diff mean_diff  std_diff   
drifting_gratings_contrast  0.020331  0.006180  0.011038  0.002087   
flashes                     0.056593  0.001947  0.023280  0.013036   
gabors                      0.020683  0.000764  0.011248  0.000518   
drifting_gratings           0.040995  0.000035  0.016981  0.002640   
natural_movie_one_shuffled  0.041651  0.000000  0.028307  0.000000   
natural_movie_three         0.255063  0.025988  0.164614  0.025418   
natural_movie_one           0.354134  0.000000  0.268361  0.000000   
static_gratings             0.188198  0.008091  0.104369  0.006266   

                                    0.3                 
                                  0.030                 
                               mean_diff      std_diff  
drifting_gratings_contrast   5142.181075  12210.187434  
flashes                      6878.501593  20599.063719  
gabors                       2348.384770   2602.262667  
drifting_gratings           33741.726118  23422.960950  
natural_movie_one_shuffled  33104.784151   1980.097972  
natural_movie_three         16233.146243   8018.927869  
natural_movie_one           13952.878647   7192.782971  
static_gratings             22060.757016  23623.524651  

[8 rows x 74 columns]

## Stimulus differentiation as a function of window and state length

In [33]:
stimulus = 'flashes'
_s = diff_all.loc[stimulus].swaplevel(1, 2)
f, ax = plt.subplots(1, 1, figsize=(5, 2.5), tight_layout=True)
ax.errorbar(
    _s[3].mean_diff.index, _s[3].mean_diff, yerr=_s[3].std_diff,
    label='3s', fmt='-o', ms=3, lw=0.75
)
ax.errorbar(
    _s[9].mean_diff.index, _s[9].mean_diff, yerr=_s[9].std_diff,
    label='9s', fmt='-o', ms=2, lw=0.75
)
ax.errorbar(
    _s[30].mean_diff.index, _s[30].mean_diff, yerr=_s[30].std_diff,
    label='30s', fmt='-o', ms=2, lw=0.75
)
ax.set_xlabel('state length (s)')
ax.set_ylabel('differentiation')
ax.legend(fontsize=8, title='window length', title_fontsize=9)
ax.set_title(stimulus)
ax.set_xscale('log');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
window_length = 30
f, ax = plt.subplots(1, 1, figsize=(6, 2.5), tight_layout=True)
for stimulus in filenames.keys():
    _s = diff_all.loc[stimulus].swaplevel(1, 2)
    ax.errorbar(
        _s[window_length].mean_diff.index,
        _s[window_length].mean_diff,
        yerr=_s[window_length].std_diff,
        label=stimulus, c=stim_colors[stimulus]
    )
ax.set_xlabel('state length (s)')
ax.set_ylabel('differentiation')
ax.legend(fontsize=8, title='stimulus', title_fontsize=8, loc=(1.01, 0))
ax.set_title(f'{window_length}s window length')
ax.set_yscale('log', nonpositive='mask')
ax.set_xscale('log');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Differentiation for different stimuli

In [35]:
state_length = 0.005
f, ax = plt.subplots(1, 1, figsize=(6, 3), tight_layout=True)
for win_length in [3, 9, 30]:
    if state_length < win_length:
        ax.bar(
            np.arange(len(diff_all.index))-0.2+0.2*[3, 9, 30].index(win_length),
            diff_all[win_length][state_length]['mean_diff'],
            yerr=diff_all[win_length][state_length]['std_diff'],
            width=0.2, label=f'{win_length}s'
        )
ax.set_xticks(range(len(diff_all.index)))
ax.set_xticklabels([s.replace('_', '\n') for s in diff_all.index])
ax.legend(fontsize=7, title='window length', title_fontsize=9)
ax.set_title(f'{state_length}s state length')
ax.set_ylabel('differentiation');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Differentiation vs state length panels

In [36]:
f = plt.figure(figsize=(5, 3), )
gs = plt.GridSpec(2, 3, figure=f, hspace=0.9, wspace=0.2, height_ratios=[1, 0.8])
# gs.update(left=0.15, right=0.95, top=0.965, bottom=0.1)
ax_ac = f.add_subplot(gs[0, :])
axes = []
axes.append(f.add_subplot(gs[1, 0]))
axes.append(f.add_subplot(gs[1, 1], sharex=axes[0]))
axes.append(f.add_subplot(gs[1, 2], sharex=axes[0]))

autocorr_times = autocorr_times[autocorr_times.mean().sort_values().index]
sns.boxplot(
    data=autocorr_times, ax=ax_ac,
    linewidth=0.5, showfliers=False, color=cm.Greys(0.4, 0.4)
)
ax_ac.set_xticklabels(
    [x.get_text().replace('_', '\n') for x in ax_ac.get_xticklabels()],
    fontsize=7
)
ax_ac.set_ylabel('characteristic\ntimescale (s)', fontsize=7.8)
ax_ac.set_yticks([0, 0.5, 1, 1.5])
ax_ac.tick_params(axis='both', labelsize=7)

example_stimuli = [
    'natural_movie_one_shuffled',
    'static_gratings', 'flashes'
]
xcoords = [0.07, 0.55, 0.815]
ycoords = [-0.6, -0.4, -0.3]

for i, (ax, stim) in enumerate(zip(axes, example_stimuli)):
    _d = diff_all.loc[stim].groupby(level=[1, 2]).mean().unstack()
    _d.mean_diff.plot(ax=ax, yerr=_d.std_diff, lw=0.75)
    if i==0:
        ax.set_ylabel('stimulus\ndifferentiation', fontsize=8)
    ax.axvline(autocorr_times[stim].mean(), c='grey', lw=0.6)
    ax.set_xscale('log')
    ax.set_yscale('log', nonpositive='mask')
    ax.tick_params(axis='both', labelsize=7, pad=0.5)
    ax.set_ylim(0.01, 1)
    ax.set_xticks([0.01, 0.1, 1, 10])
    sns.despine(ax=ax)
    if i!=0:
        ax.set_yticks([])
    patch = patches.ConnectionPatch(
        xyA=(xcoords[i], ycoords[i]),
        xyB=(0.4, 1),
        coordsA="axes fraction",
        coordsB="axes fraction",
        axesA=ax_ac,
        axesB=ax,
        arrowstyle="->",
        clip_on=False,
        color=cm.Greys(0.6, 0.9)
    )
    ax_ac.add_artist(patch)
    
axes[1].set_xlabel('time (s)', fontsize=8, labelpad=2)
f.align_ylabels()

f.savefig('fig_timescales_stim_ac.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'autocorr_times' is not defined

In [35]:
f.savefig('fig_timescales_stim_ac.pdf')

# Generate required movies

## Gabors

In [14]:
params = pd.read_csv(path.join(stimulus_directory, 'gabors', 'index.csv'))
params

,contrast,orientation,pos_x,pos_y,spatial_frequency,temporal_frequency
0,0.8,45.0,40.0,10.0,0.08,4.0
1,0.8,0.0,40.0,-40.0,0.08,4.0
2,0.8,90.0,-20.0,0.0,0.08,4.0
3,0.8,0.0,30.0,-40.0,0.08,4.0
4,0.8,45.0,10.0,30.0,0.08,4.0
...,...,...,...,...,...,...
238,0.8,90.0,0.0,0.0,0.08,4.0
239,0.8,90.0,20.0,40.0,0.08,4.0
240,0.8,45.0,30.0,40.0,0.08,4.0
241,0.8,45.0,30.0,0.0,0.08,4.0


In [42]:
# load pre-generated gabor frames (192x130 pixels)
def gen_gabor_frame(idx):
    return np.load(path.join(stimulus_directory, 'gabors', f'{idx}.npy'))[0]

def gen_gabor_clip(t_length_s=90):
    conditions = np.random.randint(0, len(params), t_length_s*4) # each condition is for 1/4 seconds
    frames = np.zeros((t_length_s*frame_rate, *frame_size))
    for t in tqdm(range(t_length_s*frame_rate)):
        i = int(t/frame_rate*4)
        frames[t] = gen_gabor_frame(conditions[i])
    return frames

In [43]:
T = 90
try:
    display(Video(path.join(stimulus_directory, f'stim_gabors_{T}.mp4'), width=frame_size[1]*2, height=frame_size[0]*2))
except:
    clip = gen_gabor_clip(T)
    np.save(path.join(stimulus_directory, f'stim_gabors_{T}.npy'), clip, allow_pickle=False)
    with capture_output():
        imageio.mimwrite(path.join(stimulus_directory, f'stim_gabors_{T}.mp4'), clip.astype('uint8'), fps=frame_rate)
    display(Video(path.join(stimulus_directory, f'stim_gabors_{T}.mp4'), width=frame_size[1]*2, height=frame_size[0]*2))

## Drifting gratings

In [44]:
params = pd.read_csv(path.join(stimulus_directory, 'drifting_gratings', 'index.csv'))
params_dgc = pd.read_csv(path.join(stimulus_directory, 'contrast_response', 'index.csv'))
display(params.head())
display(params_dgc.head())

,contrast,orientation,spatial_frequency,temporal_frequency
0,0.8,315.0,0.04,15.0
1,0.8,135.0,0.04,4.0
2,0.8,225.0,0.04,1.0
3,0.8,180.0,0.04,8.0
4,0.8,0.0,0.04,8.0


,contrast,orientation,spatial_frequency,temporal_frequency
0,1.00,0.0,0.04,2.0
1,0.60,90.0,0.04,2.0
2,0.60,135.0,0.04,2.0
3,1.00,45.0,0.04,2.0
4,0.01,45.0,0.04,2.0


In [45]:
# generate a drifting grating cliplet for a single parameter set
# drifting gratings contrast has a flash time of 250ms and regular dg has 2s
def gen_dg_cliplet(idx, stim='dg'):
    if stim=='dg':
        flash_time=2000
        clip_time=3000
    elif stim=='dgc':
        flash_time=250
        clip_time=1000
    frames = np.zeros((int(clip_time/1000*frame_rate), *frame_size))+127
    if stim=='dg':
        contrast, orientation, s_freq, t_freq = params.loc[idx]
    elif stim=='dgc':
        contrast, orientation, s_freq, t_freq = params_dgc.loc[idx]
    s_freq = s_freq*s_freq_scale
    theta = orientation/180*np.pi
    for i in range(int(flash_time/1000*frame_rate)): # 2s worth of drifting gratings
        phase = 2*np.pi*t_freq*i/frame_rate
        for x in range(len(frames[i])):
            for y in range(len(frames[i][x])):
                sin_profile = np.cos(
                    phase+(y*np.cos(theta)-x*np.sin(theta))*s_freq
                )
                try:
                    frames[i][x][y] = 127+int(127*contrast*sin_profile)
                except Exception:
                    print('Error occurred with drifting grating', params if stim=='dg' else params_dgc)
                    frames[i][x][y] = 127
    return frames

def gen_dg_clip(t_length_s=90, stim='dg'):
    if stim=='dg':
        flash_time=2000
        clip_time=3000
    elif stim=='dgc':
        flash_time=250
        clip_time=1000
    if stim=='dg':
        conditions = np.random.randint(0, len(params), int(t_length_s/(clip_time/1000)))
    elif stim=='dgc':
        conditions = np.random.randint(0, len(params_dgc), int(t_length_s/(clip_time/1000)))
    frames = np.zeros((t_length_s*frame_rate, *frame_size))+127
    for i, c in enumerate(tqdm(range(len(conditions)), desc=stim)):
        t = int(c*frame_rate*clip_time/1000)
        frames[t:t+int(clip_time/1000*frame_rate)] = gen_dg_cliplet(conditions[i], stim=stim)
    return frames

In [46]:
# # Test generation of a single grating clip
# cliplet = gen_dg_cliplet(2, stim='dgc')
# with capture_output():
#     imageio.mimwrite('stim_tmp.mp4', cliplet.astype('uint8'), fps=frame_rate)
# display(Video('stim_tmp.mp4', width=frame_size[1], height=frame_size[0]))

In [47]:
T = 90
for stim in ['dgc', 'dg']:
    try:
        display(Video(path.join(stimulus_directory, f'stim_{stim}_{T}.mp4'), width=frame_size[1]*2, height=frame_size[0]*2))
    except:
        clip = gen_dg_clip(T, stim=stim)
        np.save(path.join(stimulus_directory, f'stim_{stim}_{T}.npy'), clip, allow_pickle=False)
        with capture_output():
            imageio.mimwrite(path.join(stimulus_directory, f'stim_{stim}_{T}.mp4'), clip, fps=frame_rate)
        display(Video(path.join(stimulus_directory, f'stim_{stim}_{T}.mp4'), width=frame_size[1]*2, height=frame_size[0]*2))

## Static gratings

In [48]:
params = pd.read_csv(path.join(stimulus_directory, 'static_gratings', 'index.csv'))
params

,contrast,orientation,spatial_frequency,phase
0,0.8,120.0,0.02,0.00
1,0.8,120.0,0.32,0.75
2,0.8,30.0,0.04,0.00
3,0.8,60.0,0.32,0.75
4,0.8,30.0,0.16,0.75
...,...,...,...,...
116,0.8,90.0,0.08,0.50
117,0.8,0.0,0.08,0.25
118,0.8,30.0,0.02,0.00
119,0.8,120.0,0.16,0.50


In [49]:
# load pre-generated gabor frames (192x130 pixels)
def gen_sg_frame(idx):
    return np.load(path.join(stimulus_directory, 'static_gratings', f'{idx}.npy'))[0]

def gen_sg_clip(t_length_s=90):
    conditions = np.random.randint(0, len(params), t_length_s*4) # each condition is for 1/4 seconds
    frames = np.zeros((t_length_s*frame_rate, *frame_size))
    for t in tqdm(range(t_length_s*frame_rate)):
        i = int(t/frame_rate*4)
        frames[t] = gen_sg_frame(conditions[i])
    return frames

In [50]:
T = 90
try:
    display(Video(path.join(stimulus_directory, f'stim_sg_{T}.mp4'), width=frame_size[1]*2, height=frame_size[0]*2))
except:
    clip = gen_sg_clip(T)
    np.save(path.join(stimulus_directory, f'stim_sg_{T}.npy'), clip, allow_pickle=False)
    with capture_output():
        imageio.mimwrite(path.join(stimulus_directory, f'stim_sg_{T}.mp4'), clip.astype('uint8'), fps=frame_rate)
    display(Video(path.join(stimulus_directory, f'stim_sg_{T}.mp4'), width=frame_size[1]*2, height=frame_size[0]*2))

## Flashes
250ms display followed by 1750ms grey screen

In [51]:
params = pd.read_csv(path.join(stimulus_directory, 'flashes', 'index.csv'))
params

,color,contrast
0,1.0,0.8
1,-1.0,0.8


In [52]:
# load pre-generated gabor frames (192x130 pixels)

def gen_flashes_clip(t_length_s=90):
    conditions = np.random.randint(0, 2, int(t_length_s/2))
    frames = np.zeros((t_length_s*frame_rate, *frame_size))+127
    for i, c in enumerate(conditions):
        frames[i*2*frame_rate:i*2*frame_rate+int(0.25*frame_rate)] = int(127 * (1 + (c-0.5)*2*0.8))
    return frames

In [53]:
T = 90
try:
    display(Video(path.join(stimulus_directory, f'stim_flash_{T}.mp4'), width=frame_size[1]*2, height=frame_size[0]*2))
except:
    clip = gen_flashes_clip(T)
    np.save(path.join(stimulus_directory, f'stim_flash_{T}.npy'), clip, allow_pickle=False)
    with capture_output():
        imageio.mimwrite(path.join(stimulus_directory, f'stim_flash_{T}.mp4'), clip.astype('uint8'), fps=frame_rate)
    display(Video(path.join(stimulus_directory, f'stim_flash_{T}.mp4'), width=frame_size[1]*2, height=frame_size[0]*2))

## Movies

In [54]:
# downsample in space and upsample in time

def process_movie(movie_name, shuffle=False):
    try:
        display(Video(path.join(stimulus_directory, f'stim_{movie_name}{"_shuffled" if shuffle else ""}.mp4'), width=frame_size[1]*2, height=frame_size[0]*2))
        return
    except Exception as e:
        print(e)
    movie_path = path.join(stimulus_directory, 'touch_of_evil', f'{movie_name}.npy')
    movie = np.load(movie_path)
    if shuffle:
        np.random.shuffle(movie)
    # crop movie
    orig_fs = movie.shape[1:]
    x_factor = int(orig_fs[1]/frame_size[1])
    y_factor = int(orig_fs[0]/frame_size[0])
    x_crop = int((orig_fs[1] - x_factor*frame_size[1])/2)
    y_crop = int((orig_fs[0] - y_factor*frame_size[0])/2)
    cropped_movie = movie[:, y_crop:-y_crop, x_crop:-x_crop]

    # block average
    block_averaged_movie = cropped_movie.reshape(
        cropped_movie.shape[0],
        cropped_movie.shape[1],
        int(cropped_movie.shape[2]/x_factor),
        x_factor
    ).mean(axis=-1).transpose(0, 2, 1).reshape(
        cropped_movie.shape[0],
        192,
        int(cropped_movie.shape[1]/y_factor),
        y_factor
    ).mean(axis=-1).transpose(0, 2, 1)

    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3), tight_layout=True)
    ax1.imshow(movie[0], cmap=cm.Greys_r)
    ax2.imshow(block_averaged_movie[0], cmap=cm.Greys_r)
    ax1.set_title('original')
    ax2.set_title('downsampled')
    for ax in [ax1, ax2]:
        ax.axis('off')

    upsampled_movie = resize(block_averaged_movie, (block_averaged_movie.shape[0]/30*frame_rate, *block_averaged_movie.shape[1:]))
    np.save(path.join(stimulus_directory, f'stim_{movie_name}{"_shuffled" if shuffle else ""}.npy'), upsampled_movie, allow_pickle=False)
    with capture_output():
        imageio.mimwrite(path.join(stimulus_directory, f'stim_{movie_name}{"_shuffled" if shuffle else ""}.mp4'), upsampled_movie.astype('uint8'), fps=frame_rate)
    display(Video(path.join(stimulus_directory, f'stim_{movie_name}{"_shuffled" if shuffle else ""}.mp4'), width=frame_size[1]*2, height=frame_size[0]*2))

In [55]:
process_movie('natural_movie_one')
process_movie('natural_movie_one', shuffle=True)
process_movie('natural_movie_three')

name 'x' is not defined


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

name 'x' is not defined


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

name 'x' is not defined


/home/saurabh.gandhi/Projects/.env_difflib/lib/python3.6/site-packages/ipykernel_launcher.py:35: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Natural scenes
We do not have the scenes data yet

## Dot motion
Not implemented